In [7]:
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.metrics import root_mean_squared_error, r2_score
import pandas as pd

In [8]:
train_df: pd.DataFrame = pd.read_pickle('train_df_20241121_223002.pkl')
val_df: pd.DataFrame = pd.read_pickle('val_df_20241121_223002.pkl')
test_df: pd.DataFrame = pd.read_pickle('test_df_20241121_223002.pkl')

df = pd.concat([train_df, val_df, test_df])

print(df.select_dtypes(include=['object']).columns)

x = df.drop(columns=['ConvertedCompYearly', 'MainBranch', 'RemoteWork', 'DevType', 'Country', 'AISelect'])
y = df['ConvertedCompYearly']

Index(['MainBranch', 'RemoteWork', 'DevType', 'Country', 'AISelect'], dtype='object')


In [9]:
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)

In [15]:
for train_index, test_index in k_fold.split(x):
    X_train, X_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    pca = PCA(n_components=0.7)

    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    model = LinearRegression()
    model.fit(X_train_pca, y_train)

    y_pred = model.predict(X_test_pca)

    rmse = root_mean_squared_error(y_test, y_pred)
    print(rmse)

    r2 = r2_score(y_test, y_pred)
    print(r2)



152044.49072554117
0.16604936385678626
101874.75384803163
0.2547890795507253
214727.85853923814
0.09661229087965117
248764.8992429683
0.07719221748047234
127273.67390091762
0.2001038805963642
